In [23]:
def parse(driver,l):
    results = []
    count = [0]*len(l)
    
    #TITLE
    titleStr = driver.find_element_by_id("title").text

    #print(titleStr)

    for x in l:
        if x in titleStr:
            count[l.index(x)]+=titleStr.count(x)
            results.append(x)
        
        
    #ABSTRACT
    abstractStr = driver.find_element_by_id("abstract").text

    #print(abstractStr)

    for x in l:
        if x in abstractStr:
            count[l.index(x)]+=abstractStr.count(x)

        
        
    #CLAIMS
    claims = driver.find_elements_by_class_name("claims")

    claimList=[]

    for x in claims:
        claimList.append(x.text+"\n")
    claimStr = ""
    for x in claimList:
        claimStr+=x

    #print(claimStr)

    for x in l:
        if x in claimStr:
            count[l.index(x)]+=claimStr.count(x)

    for y in l:
        if count[l.index(y)]>=3 and y not in results:
            results.append(y)
    
    c = 0
    if len(results)==0:
        for y in l:
            if count[l.index(y)]!=0 and c<2:
                results.append(y)
                count+=1
    
    return results
        #print("Target: "+y)
        #print("#s of times seen: "+str(count[l.index(y)])+"\n")

In [24]:
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium import webdriver
import time
driver = webdriver.Firefox(executable_path=r'C:\Users\jjohn\Desktop\Scraping\geckodriver-v0.29.1-win64\geckodriver.exe') #make sure this exists somewhere in a local, varies from user to user, and copy the path here

driver.get("https://patents.google.com/patent/US8536308B2/en?q=IL6&oq=IL6")

time.sleep(5)

#GENE ARRAY
l = ["TP53","TNF","EGFR","VEGFA","APOE","IL6","TGFBI","MTHFR","ESR1","AKT1"]

list = parse(driver,l)

print("Genes found")
for x in list:
    print(x)

driver.close()

Genes found
